In [1]:
import os
# import sys
# sys.path.append('/Users/cmagri1/OneDrive - Johns Hopkins/git')
# from EncodingModel_cm296 import utils as emutils
# from EncodingModel_cm296 import cv_regression as cv

# from EncodingModel_cm296 import feature_extractor as emfe
# from DataVisualizations_cm296 import scatterplots as emscatter
# from DataVisualizations_cm296 import utils as dvutils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utilsCM


from tqdm import tqdm

In [2]:
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../Code-001/'
Y_embeddings = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
Y_embeddings = Y_embeddings.values[:,1:Y_embeddings.shape[1]-1].astype(np.float)

In [34]:
keyword = {'ROIpred'}
# keyword = {'DNNActvtn'}

# layer =  {'conv_1', 'conv_2','conv_3','conv_4','conv_5'}
layer =  {'conv_5'}

# ROI = {'EVC','LOC','PFS','OPA','PPA','RSC','FFA','OFA','STS','EBA'}
ROI = {'EVC','LOC','PPA'}
# ROI = {'EVC','PPA'}

Sub = [1,2,3,4]

#how many components to keep?
Keepncomps = [40]
# Keepncomps = [10,40,80]

# kf = KFold(n_splits = 9)


In [35]:
for ilayer in layer:
    for ikeyword in keyword:
        
        if ikeyword is 'ROIpred':
            
            for iROI in ROI:
                
                predictor_variable = {}
                for iSub in Sub:
                    thisSub = np.load(os.getcwd() + '/data-00/' +  "ROIpred_Sub" + str(iSub) + '_' + iROI + "_" + ilayer + '.npy')
                    #load ROIpred as predictor variable
                    if iSub is 1:
                        predictor_variable = thisSub
                    else:
                        predictor_variable = np.append( predictor_variable , thisSub, axis = 1)

                for icomps in Keepncomps:
                    mean_r = {}
                    mean_r = utilsCM.iter_cvregress(predictor_variable,Y_embeddings,ikeyword,ilayer,icomps,iROI,datapath='data-01/')
#                     utilsCM.make_figure(mean_r,ikeyword,ilayer,icomps,iROI, figurepath = 'figures-01/')
#                     print(mean_r)
        
        
        elif ikeyword is 'DNNActvtn':
            predictor_variable = pd.read_csv(os.getcwd() + '/data-00/' +  "things_" + ilayer + '.csv', header=None, index_col=0).iloc[:,:].to_numpy()
            for icomps in Keepncomps:
                mean_r = {}
                mean_r = utilsCM.iter_cvregress(predictor_variable,Y_embeddings,ikeyword,ilayer,icomps,datapath='data-01/')
#                 utilsCM.make_figure(mean_r,ikeyword,ilayer,icomps, figurepath = 'figures-01/')

k-fold regression, independet variable: 40 PCs retained of ROIpred from LOC


/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/C

k-fold regression, independet variable: 40 PCs retained of ROIpred from EVC


/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/C

k-fold regression, independet variable: 40 PCs retained of ROIpred from PPA


/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/utilsCM.py:132: RuntimeWarning: invalid value encountered in true_divide
  zs = lambda v: (v - v.mean(0)) / v.std(0)
/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/C

In [36]:
filename_PPA = 'PredictSENSES_' + 'ROIpred' + '_'+ 'PPA'+'_'+ ilayer+ '_'+ str(icomps) +'PCs'
thisPrediction_PPA = np.load(os.getcwd() + '/data-01/' + filename_PPA + '.npy')

filename_EVC = 'PredictSENSES_' + 'ROIpred' + '_'+ 'EVC'+'_'+ ilayer + '_'+ str(icomps) +'PCs'
thisPrediction_EVC = np.load(os.getcwd() + '/data-01/' + filename_EVC + '.npy')

filename_LOC = 'PredictSENSES_' + 'ROIpred' + '_'+ 'LOC'+'_'+ ilayer + '_'+ str(icomps) +'PCs'
thisPrediction_LOC = np.load(os.getcwd() + '/data-01/' + filename_LOC + '.npy')

# filename_DNN = 'PredictSENSES_' + 'DNNActvtn' +'_'+ ilayer + '_'+ str(icomps) +'PCs'
# thisPrediction_DNN = np.load(os.getcwd() + '/data-01/' + filename_DNN + '.npy')

In [37]:
thisPrediction_PPA

array([0.16496019, 0.10114301, 0.12379206, ..., 0.12305212, 0.03688815,
       0.09418609])

In [38]:
thisPrediction_EVC

array([0.1727825 , 0.15557366, 0.11384222, ..., 0.12834745, 0.06006222,
       0.09842186])

In [39]:
thisPrediction_LOC

array([0.17595037, 0.15143328, 0.09116192, ..., 0.15350086, 0.09900339,
       0.08840063])

In [ ]:
thisPrediction_DNN

In [ ]:
thisPrediction_PPA == thisPrediction_LOC

In [ ]:
np.float64(thisPrediction_PPA[0])

In [ ]:
np.float64(thisPrediction_LOC[0])

In [ ]:
np.float64(thisPrediction_EVC[0])

In [ ]:
from sklearn.linear_model import Ridge
import numpy as np

def regression_iter(x_train, y_train, x_test, y_test, l2=0.0, validate=True):
    regr = Ridge(alpha=l2, fit_intercept=False)
    regr.fit(x_train, y_train)
    weights = regr.coef_
#     print(weights[0,0:3])
    if validate:
        y_pred = regr.predict(x_test)
        print('shape ypred ' , y_pred.shape)
        print('shape ypred transpose' , y_pred.transpose().shape)
#         print('shape ytest ' , y_test.shape)
        r = correlation_iter(y_test, y_pred)
#         print(r)
        return weights,r
    else:
        return weights
    
def correlation_iter(a, b):
    zs = lambda v: (v - v.mean(0)) / v.std(0)
    r = (zs(a) * zs(b)).mean(axis = 0)

    return r


# def correlation_iter_nonzero(a, b):
    
#     ##Of denominator is zero, approximate it to adjust division
#     a_std = a.std(0)
#     a_std[a_std==0] = 1e-6;
    
#     b_std = b.std(0)
#     b_std[b_std==0] = 1e-6;
    
#     zs_a =  (a - a.mean(0)) / a_std
#     zs_b =  (b - b.mean(0)) / b_std
    
#     r = (zs_a * zs_b).mean(axis = 0)
    
# #     print('at least one aSTD is zero: ', (a_std==0).any() ) 

# #     print('at least one bSTD is zero: ', (b_std==0).any())  
    
# #     print('Correlation: ', r)
#     return r